In [ ]:
# Importem llibreries necessaries per implementar el mètode Màquines de Suport Vectorial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import openpyxl
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [ ]:
#Carreguem fitxer Excel i el llegim per poder ser utilitzat
xls = ''
fitxer = openpyxl.load_workbook(xls)

df = pd.read_excel(xls, sheet_name = '', engine = 'openpyxl')

In [ ]:
# Mostrem les 5 primeres files per conèixer com és el fitxer
df.head()

In [ ]:
# Agrupem per la variable Target per saber com d'equilibrat està el conjunt de dades
df[''].value_counts()

In [ ]:
# Bucle per poder representar tots els Histogrames (de cada variable) en relació a la quantitat que hi ha de cada valor 
# de la variable
variables = len(df.columns)

heads = np.empty((1,variables), dtype='object')
j=0

for col in df.columns:
    heads[0,j] = col
    j+=1

print: heads

for i in range(variables-1):
    
    intervals = range( int(np.min(df.iloc[:,i])) , int(np.max(df.iloc[:,i])) + 2 )

    #Creem el gràfic en Seaborn
    plt.figure(figsize=[20,50])
    sb.displot(df.iloc[:, i], color='#F2AB6D', bins=intervals, kde=True) 

    #Fem la configuració amb Matplotlib
    #plt.xticks(intervals)
    plt.ylabel('Frequència')
    plt.xlabel(heads[0,i])
    plt.show()

In [ ]:
# Mirem de quin tipus són les característiques (int, float, object...) per si s'han de realitzar més modificacions
df.info()

In [ ]:
#Si hi ha algunes variables simboliques (objecte) s'han de modificar per ser numèriques

In [ ]:
# Definim vector X,volem totes les files i totes les columnes menys la del target
X = df.iloc[:, :-1] 
X.head()

In [ ]:
# Definim vector y, volem totes les files i només l'última columna
y = df.iloc[:, -1] 
y.head()

In [ ]:
# Calculem la correlació entre les diferents variables
correlacio = df.corr(method='pearson')
print(correlacio)

In [ ]:
#Primera opció 75% de dades per a entrenar i 25% per a test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state= 150) 
print('Tren entrenament: ', X_train.shape, y_train.shape)
print('Tren test: ', X_test.shape, y_test.shape)

In [ ]:
# Escalem els valors ja que estan en escales diferents i el mètode no pren el mateix pes per a cada variable, entén que les que
# tenen el valor més gran, tenen més pes. 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_array = sc.fit_transform(X_train.values)
X_train = pd.DataFrame(X_train_array, index=X_train.index, columns=X_train.columns)
X_test_array = sc.transform(X_test.values)
X_test = pd.DataFrame(X_test_array, index=X_test.index, columns=X_test.columns)

In [ ]:
#Entrenem el model sense ajustar els paràmetres
model_1 = SVC()
model_1.fit(X_train, y_train)

#predicció de resultats
prediccio = model_1.predict(X_test)
print(classification_report(y_test, prediccio))

#Podem veure que 89% de precisió però pels casos de 0, és a dir, el classificador sempre classifica tot en una mateixa classe, 
# (la classe 0). Necessitem ajustar els paràmetres perquè aconsegueixi classificar en la classe 1. 

In [ ]:
#Fem la matriu de confusió per veure la quantitat de 0 / 1 que el model és capaç d'encertar
confusion_matrix(y_test, prediccio)

In [ ]:
# Calculem accuracy del model
accuracy_score(y_test, prediccio)

In [ ]:
#Provem amb els diferents tipus que existeixen (Radial, lineal, polinomial, sigmoidal)
#Cada tipus diferent te uns parametres diferents
#Paràmetres: c (per tots), degree (poly), gamma (poly, sigmoid, rbf), coef0 (poly, sigmoid)

In [ ]:
#Definim el rang dels parametres que necessitarem en cada cas
trials = 5
C_array = np.zeros(trials)
Gamma_array = np.zeros(trials)
deg_array = np.zeros(trials, dtype= 'int')
coef0_array = np.zeros(trials)

for i in range(trials):
    if (i == 0):
        C_array[i] = 0.0001
        Gamma_array[i] = 0.0001
        deg_array[i] = 1
        coef0_array[i] = 0.0001
    
    elif(i % 2 == 0):
        C_array[i] = C_array[i-1]*2
        Gamma_array[i] = Gamma_array[i-1]*2
        deg_array[i] = deg_array[i-1]*2
        coef0_array[i] = coef0_array[i-1]*2
    else:
        C_array[i] = C_array[i-1]*5
        Gamma_array[i] = Gamma_array[i-1]*5
        deg_array[i] = deg_array[i-1]*5
        coef0_array[i] = coef0_array[i-1]*5

print("C_array: ", C_array)
print("Gamma_array: ", Gamma_array)
print("Deg_array: ", deg_array)
print("coef0_array: ", coef0_array)

In [ ]:
# A partir d'aqui s'implementen totes les funcions (Radial, Lineal, Polinomial i Sigmoide) amb els paràmetres trobats
# en el bloc anterior i es calculen mètriques per veure quin és el rendiment del model

In [ ]:
# ------------------------------------------------ KERNEL RADIAL ------------------------------------------------------

param_grid_radial = {'C': C_array, 'gamma': Gamma_array, 'kernel': ['rbf']}

grid_Radial = GridSearchCV(SVC(), param_grid_radial, refit = True, verbose = 3)

grid_Radial.fit(X_train, y_train)

# Mostrem els millors valors dels parametres
print(grid_Radial.best_params_)

print(grid_Radial.best_estimator_)

In [ ]:
#Predicció model Radial
grid_predictions_R = grid_Radial.predict(X_test)
print(classification_report(y_test, grid_predictions_R))

print(grid_Radial.score(X_train, y_train))
print(grid_Radial.score(X_test, y_test))

accuracy_score(y_test, grid_predictions_R)

confusion_matrix(y_test, grid_predictions_R)

In [ ]:
# ------------------------------------------------ KERNEL LINEAL ------------------------------------------------------
param_grid_linear = {'C': C_array, 'kernel': ['linear']}

grid_Linear = GridSearchCV(SVC(), param_grid_linear, refit = True, verbose = 3)

grid_Linear.fit(X_train, y_train)

# Mostrem els millors valors dels parametres
print(grid_Linear.best_params_)

print(grid_Linear.best_estimator_)

In [ ]:
#Predicció model Lineal
grid_predictions_L = grid_Linear.predict(X_test)
print(classification_report(y_test, grid_predictions_L))

print(grid_Linear.score(X_train, y_train))
print(grid_Linear.score(X_test, y_test))

accuracy_score(y_test, grid_predictions_L)

confusion_matrix(y_test, grid_predictions_L)

In [ ]:
# ------------------------------------------------ KERNEL POLINOMIAL ------------------------------------------------------
param_grid_poly = {'C': C_array, 'gamma': Gamma_array, 'degree': deg_array, 'coef0': coef0_array, 'kernel': ['poly']}

grid_Poly = GridSearchCV(SVC(), param_grid_poly, refit = True, verbose = 3)

grid_Poly.fit(X_train, y_train)

# Mostrem els millors valors dels parametres
print(grid_Poly.best_params_)

print(grid_Poly.best_estimator_)

In [ ]:
# Predicció model polinomial
grid_predictions_P = grid_Poly.predict(X_test)
print(classification_report(y_test, grid_predictions_P))

print(grid_Poly.score(X_train, y_train))
print(grid_Poly.score(X_test, y_test))

accuracy_score(y_test, grid_predictions_P)

confusion_matrix(y_test, grid_predictions_P)

In [ ]:
# ------------------------------------------------ KERNEL SIGMOIDE ------------------------------------------------------
param_grid_sigmoid = {'C': C_array, 'gamma': Gamma_array, 'coef0': coef0_array, 'kernel': ['sigmoid']}

grid_Sigmoid = GridSearchCV(SVC(), param_grid_sigmoid, refit = True, verbose = 3)

grid_Sigmoid.fit(X_train, y_train)

# Mostrem els millors valors dels parametres
print(grid_Sigmoid.best_params_)

print(grid_Sigmoid.best_estimator_)

In [ ]:
#Predicció model Sigmoidal
grid_predictions_S = grid_Sigmoid.predict(X_test)
print(classification_report(y_test, grid_predictions_S))

print(grid_Sigmoid.score(X_train, y_train))
print(grid_Sigmoid.score(X_test, y_test))

accuracy_score(y_test, grid_predictions_S)

confusion_matrix(y_test, grid_predictions_S)